# ICD Project

## Importing libraries

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import CountVectorizer
import contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/marta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Load Dataset

In [ ]:
data = pd.read_csv('~/Downloads/icd_marta_ana_scopus.csv')
data

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,Casillo M.; Conte D.; Lombardi M.; Santaniello...,"Casillo, Mario (23026997200); Conte, Dajana (2...",23026997200; 22834021600; 57189595215; 5719787...,A Content-Based Recommender System for Hidden ...,2022,Lecture Notes in Networks and Systems,217,NaN,NaN,97,...,978-981162101-7,NaN,NaN,English,Lect. Notes Networks Syst.,Conference paper,Final,NaN,Scopus,2-s2.0-85118993536
1,Rani C.J.; Devarakonda N.,"Rani, Challapalli Jhansi (57870952400); Devara...",57870952400; 54999915100,An effectual classical dance pose estimation a...,2022,Microprocessors and Microsystems,95,NaN,104651,NaN,...,NaN,MIMID,NaN,English,Microprocessors Microsyst,Article,Final,NaN,Scopus,2-s2.0-85139595586
2,Lilan C.; Yongsheng C.,"Lilan, Chen (57213686650); Yongsheng, Chen (65...",57213686650; 6507741439,Intelligent recommendation system based on dec...,2022,Frontiers in Computational Neuroscience,16,NaN,1048047,NaN,...,NaN,NaN,NaN,English,Front. Comput. Neurosci.,Article,Final,All Open Access; Gold Open Access; Green Open ...,Scopus,2-s2.0-85142172754
3,Min H.; Yang B.; Allen D.G.; Grandey A.A.; Liu M.,"Min, Hanyi (57194326042); Yang, Baojiang (5801...",57194326042; 58019611700; 57202914326; 6603322...,Wisdom from the crowd: Can recommender systems...,2022,Personnel Psychology,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Pers. Psychol.,Article,Article in press,NaN,Scopus,2-s2.0-85144128378
4,Dahroug A.; Vlachidis A.; Liapis A.; Bikakis A...,"Dahroug, Ahmed (57205197369); Vlachidis, Andre...",57205197369; 36440453700; 53264261900; 1164101...,Using dates as contextual information for pers...,2021,Journal of Information Science,47,1,NaN,82,...,NaN,JISCD,NaN,English,J Inf Sci,Article,Final,All Open Access; Green Open Access,Scopus,2-s2.0-85073977929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,Han J.; Yamana H.,"Han, Jungkyu (56170953400); Yamana, Hayato (70...",56170953400; 7007103823,A study on individual mobility patterns based ...,2016,International Journal of Pervasive Computing a...,12,1,NaN,23,...,NaN,NaN,NaN,English,Int. J. Pervasive Comput. Commun.,Article,Final,NaN,Scopus,2-s2.0-84976339910
283,Kontiza K.; Loboda O.; Deladiennee L.; Castagn...,"Kontiza, K. (56241323100); Loboda, O. (5720398...",56241323100; 57203987002; 56344835300; 2308815...,A museum app to trigger users' reection,2018,CEUR Workshop Proceedings,2176,NaN,NaN,NaN,...,NaN,NaN,NaN,English,CEUR Workshop Proc.,Conference paper,Final,NaN,Scopus,2-s2.0-85052382796
284,Cardoso P.J.S.; Guerreiro P.; Monteiro J.; Rod...,"Cardoso, Pedro J. S. (35602693500); Guerreiro,...",35602693500; 57205268716; 35606413800; 3497329...,Applying an implicit recommender system in the...,2018,Lecture Notes in Computer Science (including s...,10908 LNCS,NaN,NaN,421,...,978-331992051-1,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85050585878
285,Herzog D.,"Herzog, Daniel (56435678200)",56435678200,Recommending a sequence of points of interest ...,2017,RecSys 2017 - Proceedings of the 11th ACM Conf...,NaN,NaN,NaN,402,...,978-145034652-8,NaN,NaN,English,RecSys - Proc. ACM Conf. Recomm. Syst.,Conference paper,Final,NaN,Scopus,2-s2.0-85030450668


### Checking for duplicate values

In [ ]:
data.duplicated().sum()

0

since the value of duplicate values(rows) is zero is not necessary to do anything

### Cheking for null values


In [ ]:
# Checking for missing values
data.isna().sum()

Authors                           39
Author full names                 39
Author(s) ID                      39
Title                              0
Year                               0
Source title                       0
Volume                           101
Issue                            215
Art. No.                         219
Page start                        90
Page end                          90
Page count                        54
Cited by                           0
DOI                               60
Link                               0
Affiliations                      41
Authors with affiliations         41
Abstract                           0
Author Keywords                   66
Index Keywords                    84
Molecular Sequence Numbers       287
Chemicals/CAS                    287
Tradenames                       287
Manufacturers                    287
Funding Details                  191
Funding Texts                    192
References                        41
C

### Remove useless columns in the context and columns with NaN values
Deleted the columns that don't have quality information for the analysis, and only bring in noise

In [ ]:
#columns_to_drop = ['Author full names', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Sponsors', 'Molecular Sequence Numbers', 'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'ISBN', 'CODEN', 'PubMed ID']
#
#data = data.drop(columns=columns_to_drop)
#data.head(4)

## **2. Preprocess the dataset**


In [ ]:


# Function to perform text preprocessing
def preprocess_text(text):
    # Check if the input is a string
    if isinstance(text, str):
        # 1. Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)

        # 2. Convert the text to lowercase
        text = text.lower()
        
        # 4. Tokenization (dividing the text to a list of words ex: 'a ana e linda ' -> ['a', 'ana', 'e' ,'linda'] )
        words = word_tokenize(text)

        
        # 5. Stopword Removal (remove very common words like 'the', 'a' etc.)
        stop_words = set(stopwords.words('english'))
        # In here he is removing the stop words from the text 
        words = [word for word in words if word.lower() not in stop_words]

    
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]
        

        
        # 5. Stemming(reduse the words to their root form ex 'running' -> 'run')
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]
        
        # Join the processed words back into a sentence
        processed_text = ' '.join(words)
        
        return processed_text
    else:
        # Return an empty string for non-string values
        return ''
    

#create a list with all the text cloumns 
text_columns = data.select_dtypes(include='object').columns.tolist()

# Apply text preprocessing to specified text columns
for col in text_columns:
    data[col] = data[col].apply(preprocess_text)
# Drop duplicate rows based on the entire row
data = data.drop_duplicates()

# Display the updated DataFrame
data.head()


,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,casillo cont lombardi santaniello troiano vale...,casillo mario 23026997200 cont dajana 22834021...,23026997200 22834021600 57189595215 5719787449...,contentbas recommend system hidden cultur heri...,2022,lectur note network system,217,,,97,...,9789811621017,,NaN,english,lect note network syst,confer paper,final,,scopu,2s2085118993536
1,rani cj devarakonda n,rani challap jhansi 57870952400 devarakonda na...,57870952400 54999915100,effectu classic danc pose estim classif system...,2022,microprocessor microsystem,95,,104651,,...,,mimid,NaN,english,microprocessor microsyst,articl,final,,scopu,2s2085139595586
2,lilan c yongsheng c,lilan chen 57213686650 yongsheng chen 6507741439,57213686650 6507741439,intellig recommend system base decis model arc...,2022,frontier comput neurosci,16,,1048047,,...,,,NaN,english,front comput neurosci,articl,final,open access gold open access green open access,scopu,2s2085142172754
3,min h yang b allen dg grandey aa liu,min hanyi 57194326042 yang baojiang 5801961170...,57194326042 58019611700 57202914326 6603322381...,wisdom crowd recommend system predict employe ...,2022,personnel psycholog,,,,,...,,,NaN,english,per psychol,articl,articl press,,scopu,2s2085144128378
4,dahroug vlachidi liapi bikaki lópeznor sacco p...,dahroug ahm 57205197369 vlachidi andrea 364404...,57205197369 36440453700 53264261900 1164101590...,use date contextu inform personalis cultur her...,2021,journal inform scienc,47,1,,82,...,,jiscd,NaN,english,j inf sci,articl,final,open access green open access,scopu,2s2085073977929


## **3.Vectorizing Data: Bag-Of-Words ou 1 gram**
- forma mais simples de representação de texto em números.
- representa as palavras num vetor, sinalizando quais estão no texto.
- Dá um resultado de 1 se estiver presente na frase e 0 se não estiver presente.

In [ ]:


column_to_vectorize = 'Title'

# Extract text data from the specified column
text_data = data[column_to_vectorize].tolist()

# Initialize the CountVectorizer with 1-grams
vectorizer = CountVectorizer(ngram_range=(1, 1))

# Fit and transform the text data
X = vectorizer.fit_transform(text_data)

# Convert the result to a DataFrame for better visualization
vectorized_data = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Display the vectorized data
print(vectorized_data.head(4))




   10th  11th  12th  13th  16th  17th  18751915  19th  1st  20  ...  workshop  \
0     0     0     0     0     0     0         0     0    0   0  ...         0   
1     0     0     0     0     0     0         0     0    0   0  ...         0   
2     0     0     0     0     0     0         0     0    0   0  ...         0   
3     0     0     0     0     0     0         0     0    0   0  ...         0   

   world  wrap  xai  year  young  zbmath  zontal  çatalhöyük  \
0      0     0    0     0      0       0       0           0   
1      0     0    0     0      0       0       0           0   
2      0     0    0     0      0       0       0           0   
3      0     0    0     0      0       0       0           0   

   回应aigc的信息资源管理学人思考  
0                  0  
1                  0  
2                  0  
3                  0  

[4 rows x 844 columns]


In [ ]:
document = data['Title']
document

0      contentbas recommend system hidden cultur heri...
1      effectu classic danc pose estim classif system...
2      intellig recommend system base decis model arc...
3      wisdom crowd recommend system predict employe ...
4      use date contextu inform personalis cultur her...
                             ...                        
282    studi individu mobil pattern base individu fam...
283                     museum app trigger user reection
284    appli implicit recommend system prepar visit c...
285    recommend sequenc point interest group user mo...
286    similar metric social network analysi content ...
Name: Title, Length: 287, dtype: object

In [ ]:
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

def preprocess_data(documents):
    
    
    # Tokenize the documents and remove stopwords
    texts = [[word for word in simple_preprocess(str(doc))] for doc in document]
    
    # Return the list of tokenized and preprocessed texts
    return texts


In [298]:
# Process the documents using the previously defined preprocessing function
processed_texts = preprocess_data(data)
processed_texts
print(processed_texts)

[['contentbas', 'recommend', 'system', 'hidden', 'cultur', 'heritag', 'site', 'enhanc'], ['effectu', 'classic', 'danc', 'pose', 'estim', 'classif', 'system', 'employ', 'convolut', 'neural', 'network', 'long', 'shortterm', 'memori', 'cnnlstm', 'network', 'video', 'sequenc'], ['intellig', 'recommend', 'system', 'base', 'decis', 'model', 'archiv', 'translat', 'task'], ['wisdom', 'crowd', 'recommend', 'system', 'predict', 'employe', 'turnov', 'destin'], ['use', 'date', 'contextu', 'inform', 'personalis', 'cultur', 'heritag', 'experi'], ['onlin', 'staypoint', 'detect', 'highfrequ', 'locat', 'updat', 'stream'], ['person', 'digit', 'ecomuseum', 'case', 'proseleusi'], ['connect', 'museum', 'citi', 'environ', 'visitor', 'perspect'], ['petrou', 'bone', 'repositori', 'ancient', 'biomolecul', 'investig', 'biosystemat', 'signal', 'sheep', 'petrou', 'bone', 'use', 'geometr', 'morphometr'], ['recstudio', 'toward', 'highlymodular', 'recommend', 'system'], ['digit', 'inform', 'manag', 'mechan', 'sport'

In [299]:
from gensim import corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_texts)

In [300]:
# Create Corpus
texts = processed_texts
texts

[['contentbas',
  'recommend',
  'system',
  'hidden',
  'cultur',
  'heritag',
  'site',
  'enhanc'],
 ['effectu',
  'classic',
  'danc',
  'pose',
  'estim',
  'classif',
  'system',
  'employ',
  'convolut',
  'neural',
  'network',
  'long',
  'shortterm',
  'memori',
  'cnnlstm',
  'network',
  'video',
  'sequenc'],
 ['intellig',
  'recommend',
  'system',
  'base',
  'decis',
  'model',
  'archiv',
  'translat',
  'task'],
 ['wisdom',
  'crowd',
  'recommend',
  'system',
  'predict',
  'employe',
  'turnov',
  'destin'],
 ['use',
  'date',
  'contextu',
  'inform',
  'personalis',
  'cultur',
  'heritag',
  'experi'],
 ['onlin', 'staypoint', 'detect', 'highfrequ', 'locat', 'updat', 'stream'],
 ['person', 'digit', 'ecomuseum', 'case', 'proseleusi'],
 ['connect', 'museum', 'citi', 'environ', 'visitor', 'perspect'],
 ['petrou',
  'bone',
  'repositori',
  'ancient',
  'biomolecul',
  'investig',
  'biosystemat',
  'signal',
  'sheep',
  'petrou',
  'bone',
  'use',
  'geometr',
  

In [301]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [349]:
from gensim.models.ldamodel import LdaModel
# Set number of topics
num_topics = 20

lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, passes=100, alpha=1.0, eta=0.001, per_word_topics=True)


In [350]:
# Print the keywords for each topic
print(lda_model.print_topics())

[(0, '0.198*"semant" + 0.163*"visitor" + 0.093*"studi" + 0.082*"path" + 0.082*"content" + 0.070*"case" + 0.070*"interact" + 0.047*"search" + 0.047*"contextu" + 0.035*"word"'), (1, '0.285*"social" + 0.143*"medium" + 0.063*"assess" + 0.048*"situat" + 0.048*"enabl" + 0.048*"represent" + 0.032*"petrou" + 0.032*"bone" + 0.032*"de" + 0.032*"relev"'), (2, '0.220*"recommend" + 0.170*"network" + 0.085*"neural" + 0.085*"deep" + 0.069*"system" + 0.057*"via" + 0.057*"movi" + 0.042*"embed" + 0.042*"tag" + 0.042*"text"'), (3, '0.376*"museum" + 0.276*"person" + 0.057*"exhibit" + 0.057*"contextawar" + 0.048*"time" + 0.038*"inclus" + 0.029*"assist" + 0.029*"platform" + 0.019*"sightse" + 0.019*"compar"'), (4, '0.396*"cultur" + 0.288*"heritag" + 0.076*"experi" + 0.065*"enhanc" + 0.059*"method" + 0.041*"larg" + 0.024*"video" + 0.018*"graph" + 0.018*"environ" + 0.012*"implement"'), (5, '0.344*"base" + 0.085*"mobil" + 0.085*"prefer" + 0.085*"smart" + 0.073*"onlin" + 0.049*"artwork" + 0.037*"individu" + 0.03

In [351]:
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score:', coherence_lda)

Coherence Score: 0.5445753595136884


In [131]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [132]:
from gensim.corpora import Dictionary

# Assuming 'processed_texts' is your list of tokenized and preprocessed documents
dictionary = Dictionary(processed_texts)


pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis

TypeError: drop() takes from 1 to 2 positional arguments but 3 were given